# Table of contents

[<h3>1. Exploratory data analysis</h3>](#1)

[<h3>2. Collaborative Recommendation System</h3>](#2)

[<h3>3. Recommendations</h3>](#3)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



The datasets contain ratings and free-text tagging activities from Dummy data . Its composition in numbers:
* 7120 ratings
* 1274 foods

<h2> Content:</h2>

**foodrating3.csv that contains ratings of foods by users:**
* userId
* foodId
* rating
* timestamp

**makanan.csv that contains food information:**
* foodId
* Nama
* Tipe





# 1. Exploratory data analysis<a class="anchor" id="1"></a>




In [ ]:
# Let's have a look at the csv-files
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
print(os.listdir("/content/drive/MyDrive/dataset/collaborative2"))

['foodrating3.csv', 'makanan.csv']


In [ ]:
rating = pd.read_csv('/content/drive/MyDrive/dataset/collaborative2/foodrating.csv')
rating.shape

(892586, 4)

In [ ]:
rating.head()

,userId,foodId,rating,timestamp
0,1,1259,3.5,02/04/2005 23:53
1,1,692,3.5,02/04/2005 23:31
2,1,756,3.5,02/04/2005 23:33
3,1,772,3.5,02/04/2005 23:32
4,1,447,3.5,02/04/2005 23:29


In [ ]:
#delete timestamp since its inrelevant
rating = rating[rating.columns.drop("timestamp")]

In [ ]:
rating.describe()

,userId,foodId,rating
count,892586.000000,892586.000000,892586.000000
mean,3531.553646,635.798500,3.552189
std,2021.186220,367.556937,1.050816
min,1.000000,0.000000,0.500000
25%,1813.000000,317.000000,3.000000
50%,3534.000000,635.000000,4.000000
75%,5261.000000,954.000000,4.000000
max,7120.000000,1273.000000,5.000000


In [ ]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892586 entries, 0 to 892585
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   userId  892586 non-null  int64  
 1   foodId  892586 non-null  int64  
 2   rating  892586 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 20.4 MB


In [ ]:
rating.isnull().sum()

userId    0
foodId    0
rating    0
dtype: int64

The dataset is clean and has no missing value.


Rating only has the ID of foods and we would like to see also the name of the foods, therefore we will later get the name out of the file makanan.csv.

**makanan.csv:**

In [ ]:
food = pd.read_csv('/content/drive/MyDrive/dataset/collaborative/makanan.csv')

In [ ]:
food.head(5)

,foodId,Nama,Tipe
0,1,Sosis Bakar,ayam-daging
1,2,Ngohiong Ayam Udang,ayam-daging
2,3,Rawon Ayam,ayam-daging
3,4,Usus Goreng Crispy,ayam-daging
4,5,Ceker Rica Rica,ayam-daging


In [ ]:
food.describe()

,foodId
count,1273.000000
mean,637.000000
std,367.627756
min,1.000000
25%,319.000000
50%,637.000000
75%,955.000000
max,1273.000000


In [ ]:
food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1273 entries, 0 to 1272
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   foodId  1273 non-null   int64 
 1   Nama    1273 non-null   object
 2   Tipe    1273 non-null   object
dtypes: int64(1), object(2)
memory usage: 30.0+ KB


makanan.csv seems to be clean

In [ ]:
food.head()

,foodId,Nama,Tipe
0,1,Sosis Bakar,ayam-daging
1,2,Ngohiong Ayam Udang,ayam-daging
2,3,Rawon Ayam,ayam-daging
3,4,Usus Goreng Crispy,ayam-daging
4,5,Ceker Rica Rica,ayam-daging


In [ ]:
# Merge both DataFrame to have also the Name of the foods
df = pd.merge(food,rating)

# Keep only the columns Nama, userId and rating
df = df[['Nama','userId','rating']]

In [ ]:
# Show the result
df.head(5)

,Nama,userId,rating
0,Sosis Bakar,14,3.5
1,Sosis Bakar,18,4.5
2,Sosis Bakar,21,4.0
3,Sosis Bakar,22,4.0
4,Sosis Bakar,24,4.0


In [ ]:
# Group the name by number of ratings to see which foods where rated the most
count_rating = df.groupby("Nama")['rating'].count().sort_values(ascending=False)
count_rating.head(10)

Nama
Sambal Goreng Kentang Udang    766
Mie Ayam Bangka                764
Nasi Goreng Magelangan         761
Tahu Bubuk                     761
Gyoza                          757
Pesmol Ikan Nila               757
Lontong Sayur Padang           755
Tongseng Sapi                  755
Sambal Cibiuk                  755
Kepiting Lada Hitam            754
Name: rating, dtype: int64

In [ ]:
# Select the foods with at least 500 ratings
r = 500
more_than_200_ratings = count_rating[count_rating.apply(lambda x: x >= r)].index

# Keep only the foods with at least 500 ratings in the DataFrame
df_r = df[df['Nama'].apply(lambda x: x in more_than_200_ratings)]

In [ ]:
# Display the count of ratings the each food
# Having only the foods with at least 500 ratings
df_r.groupby("Nama")['rating'].count().sort_values(ascending=False)

Nama
Sambal Goreng Kentang Udang    766
Mie Ayam Bangka                764
Nasi Goreng Magelangan         761
Tahu Bubuk                     761
Gyoza                          757
                              ... 
Madu Mongso                    641
Kering Tempe (Orek Tempe)      639
Nasi Liwet Sunda Teri          628
Jambal Roti Tumis              624
Cheese Tart                    619
Name: rating, Length: 1273, dtype: int64

In [ ]:
before = len(df.Nama.unique())
after = len(df_r.Nama.unique())
rows_before = df.shape[0]
rows_after = df_r.shape[0]
print(f'''There are {before} food in the dataset before filtering and {after} food after the filtering.

{before} food => {after} food
{rows_before} rows before filtering => {rows_after} rows after filtering''')

There are 1273 food in the dataset before filtering and 1273 food after the filtering.

1273 food => 1273 food
891874 rows before filtering => 891874 rows after filtering


# 2. Collaborative Recommendation System<a class="anchor" id="2"></a>

In [ ]:
# Create a matrix with userId as rows and the Name of the foods as column.
# Each cell will have the rating given by the user to the food.
# There will be a lot of NaN values, because each user hasn't watched most of the foods
foods = df_r.pivot_table(index='userId',columns='Nama',values='rating')
foods.iloc[:5,:5]

Nama,Abon Ikan Tongkol,Abon Sapi,Abon Tuna,Acar Timun,Almond Crispy
userId,,,,,
1,NaN,NaN,NaN,NaN,3.5
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,5.0,5.0
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Let's choose a famous food
food = 'Mie Ayam Bangka'

# Display the first ratings of the users for this food
foods[food].head(5)

userId
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
Name: Mie Ayam Bangka, dtype: float64

In [ ]:
#function to find correlation 
def find_corr(df_foods, food_name):
    similar_to_food = df_foods.corrwith(foods[food_name])
    similar_to_food = pd.DataFrame(similar_to_food,columns=['Correlation'])
    similar_to_food = similar_to_food.sort_values(by = 'Correlation', ascending = False)
    return similar_to_food

# 3. Recommendations <a class="anchor" id="3"></a>




## 3.1. Mie Ayam Bangka




In [ ]:
# Let's try with the second food
food_name = 'Mie Ayam Bangka'
find_corr(foods, food_name).head(5)

,Correlation
Nama,
Mie Ayam Bangka,1.000000
Ayam Kecap Jahe,0.440634
Ratatouille,0.412222
Pempek Lenggang,0.384053
Sambal Cumi,0.362979


We can see that people who ate mie ayam bangka are most likely to like also ayam kecap jahe because the correlation is highler. On the bottom are the foods which are not likely to be liked by the users.

## 3.2. Sambal Goreng Kentang Udang

In [ ]:
# Let's try with the first food
food_name = 'Sambal Goreng Kentang Udang'
find_corr(foods, food_name).head(5)

,Correlation
Nama,
Sambal Goreng Kentang Udang,1.000000
Martabak Mie,0.384575
Kue Tusuk Gigi,0.384289
Martabak Manis Keju Susu,0.376577
Kimbab,0.373212


This recommendation system using correlation is simple and nonetheless effective as a basic recommendation system. It works well to recommend foods for someone who likes one specific food.